# TCP-like GBN Implementation
TCP에서와 같이 receive window size를 N(>=1)으로 지정할 수 있어, 성능이 더 낳은 implementaion이다.
가독성이 좋고 debugging이 쉽도록 object-oriented programming해 보자.

## More on Python
### f-string

In [1]:
l = ['a', 'b', 'c']
'a list: {}'.format(l)

"a list: ['a', 'b', 'c']"

In [2]:
f'a list: {l}'

"a list: ['a', 'b', 'c']"

### Enumeration - defining constants
enum 모듈에 Enum class와 이의 subclass인 IntEnum, IntFlag 등의 class가 정의되어 있다. 이들을 상속해서 class를 정의한다.
#### Enum
목록을 열거하는데 사용한다.
- `==`, `!=`, `in`(membership operator) 가능

In [1]:
from enum import Enum
class Color(Enum):
    RED = 1
    GREEN = 2
    BLUE = 3
    
color = Color.RED
print(color)
print(repr(color))
print(color.name, color.value)
print(Color.__members__)

In [2]:
for name, member in Color.__members__.items():
    print(name, member)

RED Color.RED
GREEN Color.GREEN
BLUE Color.BLUE


In [3]:
apples = {}
apples[Color.RED] = 'red delicious'
apples[Color.GREEN] = 'granny smith'
# apples == {Color.RED: 'red delicious', Color.GREEN: 'granny smith'}
print(apples[color])
print(apples)

red delicious
{<Color.RED: 1>: 'red delicious', <Color.GREEN: 2>: 'granny smith'}


In [4]:
color is Color.RED

True

In [5]:
color == Color.RED

True

In [6]:
color in Color

True

In [7]:
color == 1   # color와 1은 다른 type이니까 같지 않다.

False

In [8]:
try:
    Color.RED < Color.BLUE    # 크기 비교는 불가능
except TypeError as e:
    print(e)

'<' not supported between instances of 'Color' and 'Color'


In [9]:
# 정수에서 Enum object creation
c = Color(2)
print(c)
try:
    d = Color(4)
except ValueError as e:
    print(e)

Color.GREEN
4 is not a valid Color


In [10]:
from enum import auto

class State(Enum):   # States
    Wait    = auto()
    Closing = auto()
    Closed  = auto()
    
State.__members__

mappingproxy({'Wait': <State.Wait: 1>,
              'Closing': <State.Closing: 2>,
              'Closed': <State.Closed: 3>})

#### IntEnum - Integer Enumeration
Enum과 int에서 상속받은 class
- 크고 작음을 비교할 수 있는 Enum
- 사칙연산 등 int type operation 사용 가능하나 의미가 없다.

In [11]:
from enum import IntEnum

class Ev(IntEnum):      # events
    Packet_Arrival  = 1
    App_Request     = 16
    TO_Retransmit   = 32
    TO_Closing      = 33
    TO_DelayedACK   = 34
    
event = Ev.TO_Closing
print(event)
print(event.name, event.value)
if event == Ev.TO_Closing:
    print(event, 'occurs')
if event >= 32:
    print('a TIMEOUT event')

Ev.TO_Closing
TO_Closing 33
Ev.TO_Closing occurs
a TIMEOUT event


주의: +, - 가능하지만 의미가 없다.

In [12]:
Ev.App_Request + 16

32

#### IntFlag
IntEnum에서 상속받은 Enum으로 bitwise operation(`&`, `|`, `~`)이 가능하다.

In [13]:
from enum import IntFlag

class Type(IntFlag):
    """Packet type definition
    """
    DATA    = 1
    ACK     = 2
    FIN     = 4
    SYN     = 8
    RST     = 16
    URG     = 32
    PSH     = 64
    
t = Type.SYN | Type.ACK
print(t)
print(t.name, t.value)
if t & Type.ACK:
    print('ACK packet')
if t & Type.SYN:
    print('SYN packet')
if Type.ACK in t:
    print('ACK packet')

Type.SYN|ACK
None 10
ACK packet
SYN packet
ACK packet


## Packet class
make_pkt(), ichksum 등 function들을 Packet class의 method로 만들어 encapulate했다.

In [14]:
from packet import Seq, srange, Type, Packet, PacketBuffer

data = b'00005 abcdefghijklmnopqrstuvwxyz\n'
next_seq = Seq(5)
sndpkt = Packet(Type.DATA, next_seq, data)
print(sndpkt)
print(sndpkt.pdu)
print(sndpkt.type)
print(sndpkt.seq)
print(repr(sndpkt.seq))
print(sndpkt.checksum)
print(sndpkt.data)

DATA 5 b'\x01\x05\xda\xd400005 abcdef'
bytearray(b'\x01\x05\xda\xd400005 abcdefghijklmnopqrstuvwxyz\n')
Type.DATA
5
Seq(5)
bytearray(b'\xda')
bytearray(b'00005 abcdefghijklmnopqrstuvwxyz\n')


In [15]:
import copy
pdu = copy.copy(sndpkt.pdu)  # deep copy
pdu[0] += 1    # cause bit error
rcvpkt = Packet(pdu)
if rcvpkt.corrupt():
    print('corrupt:', rcvpkt)
data = rcvpkt.extract()
print(data)

corrupt: ACK 5 b'\x02\x05\xda\xd400005 abcdef'
bytearray(b'00005 abcdefghijklmnopqrstuvwxyz\n')


## PacketBuffer
### GBNsend: send buffer

In [16]:
N = 16
sndbuf = PacketBuffer(N)
base = Seq(254)
next_seq = Seq(254)

In [17]:
for i in range(4):
    sndbuf[next_seq] = Packet(Type.DATA, next_seq, b'hello, ')
    next_seq += 1
print(sndbuf)

254: DATA 254 b'\x01\xfe\xb9\xe3hello, '
255: DATA 255 b'\x01\xff\xb9\xe2hello, '
0: DATA 0 b'\x01\x00\xba\xe1hello, '
1: DATA 1 b'\x01\x01\xba\xe0hello, '


In [18]:
rcvpkt = Packet(Type.ACK, Seq(0))
acknum = rcvpkt.seq
for seq in srange(base, acknum):
    del sndbuf[seq]
base = acknum
print(sndbuf)

0: DATA 0 b'\x01\x00\xba\xe1hello, '
1: DATA 1 b'\x01\x01\xba\xe0hello, '


### GBNrecv: receive buffer

In [19]:
N = 16
rcvbuf = PacketBuffer(N)
for s in [Seq(254), Seq(0), Seq(1)]:
    rcvbuf[s] = Packet(Type.DATA, s, b'hello')
print(rcvbuf)

254: DATA 254 b'\x01\xfe(\xc1hello'
0: DATA 0 b'\x01\x00)\xbfhello'
1: DATA 1 b'\x01\x01)\xbehello'


#### Retrieving packet from PacketBuffer
PacketBuffer는 dict로 구현되어 있고, 0 ~ 255 범위의 Seq type의 key로 검색할 수 있다.
해당 seq에 packet이 없으면(즉, hole이 있으면) None이 return된다.

In [20]:
for s in srange(Seq(254), Seq(254)+5):
    print(rcvbuf[s])

DATA 254 b'\x01\xfe(\xc1hello'
None
DATA 0 b'\x01\x00)\xbfhello'
DATA 1 b'\x01\x01)\xbehello'
None


Hole 바로 전까지가 순서에 맞는 것이니 data를 application으로 deliver할 수 있다.  아래에서 255 packet이 저장되면, 연속된 4개의 packet에서 extract한 data를 전달하면 된다.

In [21]:
rcvpkt = Packet(Type.DATA, Seq(255), b'world') # new packet arrival
rcvbuf[rcvpkt.seq] = rcvpkt
for s in srange(Seq(254), Seq(254)+5):
    print(rcvbuf[s])

DATA 254 b'\x01\xfe(\xc1hello'
DATA 255 b'\x01\xff\x13\xc1world'
DATA 0 b'\x01\x00)\xbfhello'
DATA 1 b'\x01\x01)\xbehello'
None


*Q. 이때 ACK number는 ?*

## Timer
여러개의 timer를 start, stop할 수 있게 Timer class가 제공되어 있다. 초기 timeout interval이 다음과 같이 지정된다.
```Python
class Ev(IntEnum):      # events
    Packet_Arrival  = 1
    App_Request     = 16
    TO_Retransmit   = 32
    TO_Closing      = 33
    TO_DelayedACK   = 34

# initial timeout interval
TO_interval= {
    Ev.TO_Retransmit: 0.2 + 5 * EXTRA_MEAN_DELAY,
    Ev.TO_Closing: 1,
    Ev.TO_DelayedACK: 0.5
              }
```

예를 들어, Ev.TO_Retransmit timer를 start, stop하려면,
```Python
timer = Timer()

timer.start_timer(Ev.TO_Retransmit) # 지정된 interval이 지나면 Ev.TO_Retransmit이 발생
timer.start_timer(Ev.TO_DelayedACK) # 0.5초 지나면 Ev.TO_Delayed을 발생

timer.stop(Ev.TO_Retransmit)
```

하나의 Timer class object 여러개의 timer 가동 가능. Timeout interval이 지나면 지정한 timeout event가 발생한다.
같은 방식으로 Ev.TO_Closing timer를 가동시킬 수 있다. 
> 주의: 각각의 thread에서 각각 독립적인 Timer object을 생성해서 사용할 수 있지만, 둘 이상의 thread에서 하나의 Timer를 공유해서 사용할 수 없다. 
### Dynamic timeout interval의 변경
```Python
intv = timer.get_intv(Ev.TO_Retrasmit)

timer.set_intv(Ev.TO_Retramit, new_intv)
```

## How to start GBNsend and GBNrecv threads
Application은 다음 open 함수로 GBNsend나 GBNrecv를 실행시킬 수 있다.

In [ ]:
def open(peer_host, N, passive=False):
    """Open GBN protocol entity

    :param peer_host: peer host name
    :param passive: for sending (default), for receiving if True
    :return: GBN thread object
    """
    assert N < Seq.MOD, "N: too big"
    if passive:
        gbn = GBNrecv((peer_host, sender_port), N)
        gbn.start()
        logging.info('app receiver starts')
    else:
        gbn = GBNsend((peer_host, receiver_port), N)
        gbn.start()
        logging.info('app sender starts')
    return gbn

#### Active open to run GBNsend thread - sender에서 사용

In [ ]:
gs = gbn.open(peer_host='localhost', N=16)
pass 
gs.close()

> `gs.close()`는 FIN을 보내달라는 request이다. Sender에서 마지막은 이를 call해야 한다. (receiver에서는 사용해선 안된다.)

#### Passive open to run GBNrecv thread - receiver에서 사용

In [ ]:
r = gbn.open(peer_host='localhost', N=16, passive=True)